In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn import ensemble

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/car data.csv')

In [ ]:
df.head()

We want to predict **Selling_Price** let's do some analysis on Selling_Price

In [ ]:
plt.figure(figsize = (10, 8))
sns.distplot(df['Selling_Price'])
plt.show()

In [ ]:
print("Skwness of Selling_Price : {}".format(df['Selling_Price'].skew()))

Selling_Price is positive skewed
and there are some error look like above 30
let's try convert it into to normal distribution

In [ ]:
plt.figure(figsize = (10, 8))
sns.distplot(df['Selling_Price'], fit=norm)
plt.show()

let's focus on missing values

In [ ]:
df.isnull().sum()

There are no missing values we are good to go !

Let's see what **Present_Price** give us 

In [ ]:
plt.figure(figsize = (10, 8))
sns.distplot(df['Present_Price'], fit=norm)
plt.show()

**Present_Price** is also Positive skewed let's draw scatter plot of Present_Price vs Selling_Price

In [ ]:
plt.figure(figsize = (10, 8))
plt.scatter( x = 'Selling_Price', y='Present_Price', data = df )
plt.show()

It's look like Selling_Price and Present Price are Highly positive correlated

In [ ]:
print("Correlation between Selling_Price and Present_Price ", df['Selling_Price'].corr(df['Present_Price']) )

Let's focus on **Fuel_Type**

In [ ]:
plt.figure(figsize = (8, 6))
sns.countplot( x = 'Fuel_Type', data=df )
plt.show()

In [ ]:
df['Fuel_Type'].value_counts().plot.pie(autopct="%.1f%%")

In [ ]:
df['Fuel_Type'].value_counts()

We have good amount of vehicle that have Petrol Fuel_Type

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(20, 8) )

sns.barplot(x = 'Fuel_Type', y = 'Selling_Price', data = df, ax = ax[0])
ax[0].set_title("Barplot of Selling_Price vs Fuel_Type", fontsize=15)

sns.stripplot(x="Fuel_Type", y="Selling_Price", data=df, ax = ax[1])
ax[1].set_title("Stripplot of Selling_Price vs Fuel_Type", fontsize=15)

plt.show()

Diesel vehicle's price is very high as compared to petrol and CNG

Let's talk about Year

In [ ]:
df['Year'].value_counts()

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(16, 12) )

sns.barplot(x = 'Year', y = 'Selling_Price', data = df, ax = ax[0])
ax[0].set_title("Barplot of Selling_Price vs Year", fontsize=15)

sns.stripplot(x="Year", y="Selling_Price", data=df, ax = ax[1])
ax[1].set_title("Stripplot of Selling_Price vs Year", fontsize=15)

plt.show()

when **Year** inreasing than **Selling_Price** also increasing for some vehicles

In [ ]:
df['Kms_Driven'].describe()

In [ ]:
sns.relplot(x="Selling_Price", y="Kms_Driven", data=df, ax = ax[0]);
plt.title("Kms_Driven vs Selling_Price", fontsize=15)
plt.show()

In [ ]:
plt.scatter(x = df['Selling_Price'], y = df['Kms_Driven'])
plt.title("Kms_Driven vs Selling_Price", fontsize=15)
plt.show()

In [ ]:
df['Seller_Type'].value_counts()

In [ ]:
f, ax = plt.subplots(3, 1, figsize=(16, 24) )

sns.barplot(x = 'Seller_Type', y = 'Selling_Price', data = df, ax = ax[0])
ax[0].set_title("Barplot of Selling_Price vs Seller_Type", fontsize=15)

sns.stripplot(x="Seller_Type", y="Selling_Price", data=df, ax = ax[1])
ax[1].set_title("Stripplot of Selling_Price vs Seller_Type", fontsize=15)

sns.boxplot(x='Seller_Type', y='Selling_Price', data = df, ax = ax[2])
ax[2].set_title("Boxplot of Selling_Price vs Seller_Type")

plt.show()

Dealer's Selling_Price is high as compared to Individual

In [ ]:
df['Transmission'].value_counts()

In [ ]:
f, ax = plt.subplots(3, 1, figsize=(16, 24) )

sns.barplot(x = 'Transmission', y = 'Selling_Price', data = df, ax = ax[0])
ax[0].set_title("Barplot of Selling_Price vs Transmission", fontsize=15)

sns.stripplot(x="Transmission", y="Selling_Price", data=df, ax = ax[1])
ax[1].set_title("Stripplot of Selling_Price vs Transmission", fontsize=15)

sns.boxplot(x='Transmission', y='Selling_Price', data = df, ax = ax[2])
ax[2].set_title("Boxplot of Selling_Price vs Transmission")

plt.show()

Automatic Transmission Selling_Price is high as compared to Manual

In [ ]:
df['Owner'].value_counts()

In [ ]:
f, ax = plt.subplots(3, 1, figsize=(16, 20) )

sns.barplot(x = 'Owner', y = 'Selling_Price', data = df, ax = ax[0])
ax[0].set_title("Barplot of Selling_Price vs Owner", fontsize=15)

sns.stripplot(x="Owner", y="Selling_Price", data=df, ax = ax[1])
ax[1].set_title("Stripplot of Selling_Price vs Owner", fontsize=15)

sns.boxplot(x='Owner', y='Selling_Price', data = df, ax = ax[2])
ax[2].set_title("Boxplot of Selling_Price vs Owner")

plt.show()

In [ ]:
print("Unique value in Car_Name : ", len(df['Car_Name'].unique()))

there are lots of unique value in Car_Name so drop it.

In [ ]:
df.drop('Car_Name', axis=1, inplace=True)

In Year lots of unique value so it may be increase error so we convert continuos year column to categorical year column

In [ ]:
df['Year'].value_counts()

In [ ]:
df['Year'].replace({2010 : 1, 2011 : 2, 2012 : 3, 2013 : 4, 2014 : 5, 2015 : 6, 2016 : 7, 2017 : 8, 2018 : 9}, inplace=True)
df.loc[(df['Year'] >= 2003) & (df['Year']<= 2009), 'Year'] = 0
df['Year'].value_counts()

Convert Categorical column to numeric column

In [ ]:
df['Fuel_Type'].replace({'Petrol':0, 'Diesel':1, 'CNG':2}, inplace=True)
df['Seller_Type'].replace({'Dealer':0, 'Individual':1}, inplace=True)
df['Transmission'].replace({'Manual':0, 'Automatic':1}, inplace=True)

In [ ]:
df['Kms_Driven'].describe()

In [ ]:
df.loc[(df['Kms_Driven'] >= 500) & (df['Kms_Driven'] <= 15000), 'Kms_Driven'] = 0
df.loc[(df['Kms_Driven'] > 15000) & (df['Kms_Driven'] <= 32000), 'Kms_Driven'] = 1
df.loc[(df['Kms_Driven'] > 32000) & (df['Kms_Driven'] <= 49000), 'Kms_Driven'] = 2
df.loc[(df['Kms_Driven'] > 49000), 'Kms_Driven'] = 3

In [ ]:
df['Kms_Driven'].value_counts()

In [ ]:
df.head()

In [ ]:
X = df.drop('Selling_Price', axis=1)
y = df['Selling_Price']

X = RobustScaler().fit_transform(X)
# y = RobustScaler().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

****Linear Regression****

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("Linear regression score : ",lr.score(X_test, y_test))
print("Mean squared error : ", mean_squared_error(y_test, y_pred))

**Ridge Regession**

In [ ]:
clf = Ridge(alpha=0.1)
clf.fit(X_train, y_train)
print("Ridge Regression Score : ", clf.score(X_test, y_test))
y_pred = clf.predict(X_test)
print("Mean squared error : ", mean_squared_error(y_test, y_pred))

**Lasso**

In [ ]:
ls = Lasso()
ls.fit(X_train, y_train)
y_pred = ls.predict(X_test)
print("Lasso Score : ", ls.score(X_test, y_test))
print("Mean squared error : ", mean_squared_error(y_test, y_pred))

**Decision Tree Regressor**

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print("Decision Tree Regressor Score : ", dt.score(X_test, y_test))
print("Mean squared error : ", mean_squared_error(y_pred, y_test))

**Gradient Boosting Regressor**

In [ ]:
params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print("Score : ", reg.score(X_test, y_test))
print("Mean squared error : ", mean_squared_error(y_pred, y_test))

**From above algorithams we can say that GradientBoostingRegressor perform best with 96% accuracy and 0.79 rmse**